In [11]:
from transformers import pipeline
from utils import *
import glob
import numpy as np
import time

In [12]:
# More models in the model hub.
model_name = "openai/clip-vit-large-patch14-336"
classifier = pipeline("zero-shot-image-classification", model = model_name)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [13]:


def classify(image,labels):
    start_t = time.time_ns()
    scores = classifier(image, 
                        candidate_labels = labels)
    end_t = time.time_ns()
    
    latency = (end_t-start_t)/1e6
    
    print(scores,latency)
    return scores,latency

In [22]:
protocol = "adult_cardiact_arrest_protocol"
protocol = "respiratory_distress_protocol"

labels_for_classification = generate_labels(protocol)

# assign directory
directory = './images/resp - assist ventilation - bvm via mask/'

accuracy = []
# iterate over files in
# that directory
for filename in glob.iglob(f'{directory}/*'):
    model_scores,model_latency = classify(filename,labels_for_classification)
    accuracy.append( {"image":filename, "results": model_scores, "latency": model_latency})

[{'score': 0.7819769978523254, 'label': 'Placing BVM mask on face'}, {'score': 0.10133299976587296, 'label': 'Inserting IV to arm'}, {'score': 0.08622653782367706, 'label': 'Attaching nebulizer'}, {'score': 0.01762593910098076, 'label': 'BVM Mask'}, {'score': 0.007422406692057848, 'label': 'Inserting IV to leg'}, {'score': 0.005415155086666346, 'label': 'Nebulizer mask'}] 1766.76578
[{'score': 0.9747033715248108, 'label': 'Placing BVM mask on face'}, {'score': 0.014650977216660976, 'label': 'Attaching nebulizer'}, {'score': 0.004635334014892578, 'label': 'Inserting IV to arm'}, {'score': 0.003119581611827016, 'label': 'Nebulizer mask'}, {'score': 0.002657809993252158, 'label': 'BVM Mask'}, {'score': 0.00023298444284591824, 'label': 'Inserting IV to leg'}] 1719.612757
[{'score': 0.9720346927642822, 'label': 'Placing BVM mask on face'}, {'score': 0.010491243563592434, 'label': 'Attaching nebulizer'}, {'score': 0.01009914930909872, 'label': 'BVM Mask'}, {'score': 0.0036670719273388386, 'l

In [23]:
avg_accuracy = []
avg_latency = []
for result in accuracy:
    print('Image: ',result['image'],'Latency(ms)',result['latency'],'Scores:',result['results'][0], )
    avg_accuracy.append(result['results'][0]['score'])
    avg_latency.append(result['latency'])
    
print("Average Accuracy:",np.average(avg_accuracy))
print("Average Latency(s):",np.average(avg_latency)/1e3)

Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_4.jpg Latency(ms) 1766.76578 Scores: {'score': 0.7819769978523254, 'label': 'Placing BVM mask on face'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_2.jpg Latency(ms) 1719.612757 Scores: {'score': 0.9747033715248108, 'label': 'Placing BVM mask on face'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_3.jpg Latency(ms) 1577.436527 Scores: {'score': 0.9720346927642822, 'label': 'Placing BVM mask on face'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_1.jpg Latency(ms) 1573.272102 Scores: {'score': 0.5940059423446655, 'label': 'Attaching nebulizer'}
Image:  ./images/resp - assist ventilation - bvm via mask/assist_bvm_5.jpg Latency(ms) 1594.008344 Scores: {'score': 0.6597080230712891, 'label': 'Placing BVM mask on face'}
Average Accuracy: 0.7964858055114746
Average Latency(s): 1.6462191020000003


EMS Procedure Engine

In [ ]:
def detect_procedure(image, protocol, classifier):
    
    labels_for_classification = generate_labels(protocol)
    scores = classifier(image, 
                    candidate_labels = labels_for_classification)
    
    
    
    
